<center>
<h1>Time Series Analysis</h1>
</center>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams["figure.figsize"] = (12,4)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 14

In [ ]:
dataset = pd.read_csv('https://raw.githubusercontent.com/deba-iitbh/datasets/main/AirPassengers.csv')
dataset['Month'] = pd.to_datetime(dataset['Month'])
indexedDataset = dataset.set_index('Month')

## Exploring the Dataset

In [ ]:
## plot graph
plt.xlabel("Date")
plt.ylabel("Number of Air Passangers")
plt.plot(indexedDataset)

In [ ]:
# Determining the rolling stattistics
rollmean = indexedDataset.rolling(window = 12).mean()
rollstd = indexedDataset.rolling(window = 12).std()
print(rollmean, rollstd)

## Checking Stationarity with rolling mean and standard Deviation

In [ ]:
# Plot rolling statistics
orig = plt.plot(indexedDataset, color ='blue', label = 'Original')
mean = plt.plot(rollmean, color ='red', label = 'Rolling Mean')
std = plt.plot(rollstd, color ='green', label = 'Rolling Std')
plt.legend(loc = 'best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block = False)

## Checking Stationarity with Augmented Dickey Fuller Statistical Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform Dickey-Fuller Test
print("Results of Diceky-Fuller Test:")
dftest = adfuller(indexedDataset['#Passengers'], autolag = 'AIC')
dfoutput = pd.Series(dftest[:4], index= ["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])

for key, value in dftest[4].items():
  dfoutput[f'Critical Value ({key})'] = value

print(dfoutput)

## Wrapping Visual and Statistical tools in a single function

In [ ]:
def test_stationarity(timeseries):
  # Determining the rolling stattistics
  movingAverage = timeseries.rolling(window = 12).mean()
  movingSTD = timeseries.rolling(window = 12).std()

  # Plot rolling statistics
  orig = plt.plot(timeseries, color ='blue', label = 'Original')
  mean = plt.plot(movingAverage, color ='red', label = 'Rolling Mean')
  std = plt.plot(movingSTD, color ='green', label = 'Rolling Std')
  plt.legend(loc = 'best')
  plt.title('Rolling Mean & Standard Deviation')
  plt.show(block = False)

  # Perform Dickey-Fuller Test
  print("Results of Diceky-Fuller Test:")
  dftest = adfuller(timeseries['#Passengers'], autolag = 'AIC')
  dfoutput = pd.Series(dftest[:4], index= ["Test Statistic", "p-value", "#Lags Used", "Number of Observations Used"])

  for key, value in dftest[4].items():
    dfoutput[f'Critical Value ({key})'] = value

  print(dfoutput)

In [ ]:
test_stationarity(indexedDataset)

## Converting Non-Stationary data to Stationary dataset
- Log
- Subtracting Simple rolling Average
- Subtracting Exponential rolling Average
- Subtracting previous value(Most Popular) with shift()
- Seasonal decomposition
- Combination of the above

### 1. Log

In [ ]:
#Estimating trend
indexedDataset_logscale = np.log(indexedDataset)
plt.plot(indexedDataset_logscale)
plt.show()

In [ ]:
test_stationarity(indexedDataset_logscale)

### 2. Differencing Simple Moving Average

In [ ]:
movingAverage = indexedDataset_logscale.rolling(window = 12).mean()
datasetLogScalMinusMovingAverage = indexedDataset_logscale - movingAverage
display(datasetLogScalMinusMovingAverage.head(12))

# Remove nan values
datasetLogScalMinusMovingAverage.dropna(inplace= True)
datasetLogScalMinusMovingAverage.head(10)

In [ ]:
test_stationarity(datasetLogScalMinusMovingAverage)

### 3. Differencing Exponential Moving Average

In [ ]:
exponentialDecayWeightedAverage = indexedDataset_logscale.ewm(halflife = 12, min_periods = 0, adjust = 0).mean()
plt.plot(indexedDataset_logscale)
plt.plot(exponentialDecayWeightedAverage, color = 'red')
plt.show()

In [ ]:
datasetLogScaleMovingExponentialDecayAverage = indexedDataset_logscale - exponentialDecayWeightedAverage
test_stationarity(datasetLogScaleMovingExponentialDecayAverage)

### 4. Differencing Previous Value

In [ ]:
datasetLogDiffShifting = indexedDataset_logscale - indexedDataset_logscale.shift()
plt.plot(datasetLogDiffShifting)
plt.show()

In [ ]:
datasetLogDiffShifting.dropna(inplace = True)
test_stationarity(datasetLogDiffShifting)

### 5. Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(indexedDataset_logscale)

trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

plt.figure(figsize = (12, 7))
plt.subplot(411)
plt.plot(indexedDataset_logscale, label = "original")
plt.legend()

plt.subplot(412)
plt.plot(trend, label = "Trend")
plt.legend()


plt.subplot(413)
plt.plot(seasonal, label = "seasonal")
plt.legend()


plt.subplot(414)
plt.plot(residual, label = "Residuals")
plt.legend()


decomposedLogData = residual
decomposedLogData.dropna(inplace = True)
test_stationarity(decomposedLogData.to_frame(name = "#Passengers"))

In [ ]:
type(decomposedLogData)

## ARMA modelling

### Auto Regressive Model

AR(1)
$$
P(today) = Mean + Coeff * P(yesterdat) + noise
$$
$$
P(t) = \mu + \gamma_1 * P(t-1) + \epsilon
$$
- If gamma_1 = 0 model is Mean plus noise

AR(2)
$$
P(t) = \mu + \gamma_1 * P(t-1) + \gamma_2 * P(t-2) \epsilon
$$

### Moving Average MA Model
MA(1) Model
$$
P(t) = \mu + \theta1 * \epsilon(t-1) + \epsilon(t)
$$

## ARMA Model
ARMA(1,1)
$$
P(t) = \mu + \gamma_1 * P(t-1) + \theta_1 * \epsilon(t-1) + \epsilon(t)
$$

ARMA(1,2)
$$
P(t) = \mu + \gamma_1 * P(t-1) + \theta_1 * \epsilon(t-1) + \theta_2 * \epsilon(t-2) + \epsilon(t)
$$

ARMA(2,1)
$$
P(t) = \mu + \gamma_1 * P(t-1) + \gamma_1 * P(t-2) + \theta_1 * \epsilon(t-1) + \epsilon(t)
$$

ARMA(2,2)
$$
P(t) = \mu + \gamma_1 * P(t-1) + \gamma_1 * P(t-2) + \theta_1 * \epsilon(t-1) + \theta_2 * \epsilon(t-2) + \epsilon(t)
$$

## Finding Lags of AR and MA models

In [ ]:
#ACF and PACF plots:
from statsmodels.tsa.stattools import acf, pacf

lag_acf = acf(datasetLogDiffShifting, nlags=20)
lag_pacf = pacf(datasetLogDiffShifting, nlags=20, method='ols')

#Plot ACF:
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0, linestyle = '--', color="gray")
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle="--",color="gray")
plt.axhline (y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle="--", color= "gray")
plt.title("Autocorrelation Function")

#Plot PACF:
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--', color="gray")
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle="--",color='gray')
plt.axhline (y=1.96/np.sqrt(len(datasetLogDiffShifting)), linestyle="--",color='gray')
plt.title('Partial Autocorrelation Function')

plt.tight_layout()

In [ ]:
from statsmodels.tsa.stattools import arma_order_select_ic
arma_order_select_ic(datasetLogDiffShifting)

## AR Model

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#AR MODEL
print("Plotting AR model")
model = ARIMA(indexedDataset_logscale, order=(2, 1, 0))
results_AR = model.fit(disp=-1)
plt.plot(results_AR. fittedvalues, color="red")
title = sum((results_AR.fittedvalues - datasetLogDiffShifting ["#Passengers"])**2)
plt.title(f"RSS: {title:.44}")
plt.show()

plt.plot(datasetLogDiffShifting)

## MA Model

In [ ]:
#MA MODEL
model = ARIMA (indexedDataset_logscale, order=(8, 1, 2))
results_MA = model.fit(disp=-1)
plt.plot(results_MA.fittedvalues, color="red")
plt.plot(datasetLogDiffShifting)
title = sum((results_MA.fittedvalues - datasetLogDiffShifting["#Passengers"])**2)
plt.title(f"RSS: {title:.44}")
plt.show()

## ARIMA

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model = ARIMA (indexedDataset_logscale, order=(2, 1, 2))
results_ARIMA = model.fit()
plt.plot(datasetLogDiffShifting)
plt.plot(results_ARIMA. fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA. fittedvalues-datasetLogDiffShifting["#Passengers"])**2))

### Getting Predictions

In [ ]:
predictions_ARIMA_diff = pd.Series (results_ARIMA. fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

Remember the predictions are in Log_differenceed values.We need to convert them to original form

In [ ]:
#Convert to cumulative sum
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum.head())

In [ ]:
predictions_ARIMA_log = pd.Series(indexedDataset_logscale["#Passengers"].iloc[0], index=indexedDataset_logscale.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum, fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(indexedDataset)
plt.plot(predictions_ARIMA)
plt.show()

In [ ]:
results_ARIMA.forecast(steps = 120)

In [ ]:
results_ARIMA.plot_predict(1,264)

## How to get better results
- Use Seasonal models
- Use Other Features
- Combining the above two tasks using SARIMAX model